In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://results.eci.gov.in/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
# URL of the website
url = "https://results.eci.gov.in/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links containing "https://results.eci.gov.in"
links = soup.find_all('a', href=True)
eci_links = [link['href'] for link in links if "https://results.eci.gov.in" in link['href']]
eci_links = set(eci_links)

# Create a dictionary with webpage names as keys
eci_links_dict = {}
for link in eci_links:
    # Extract the webpage name from the URL
    key = link.split('/')[-2] if link.endswith('/') else link.split('/')[-2]
    # Ensure unique keys by appending an index if necessary
    if key in eci_links_dict:
        i = 1
        while f"{key}_{i}" in eci_links_dict:
            i += 1
        key = f"{key}_{i}"
    eci_links_dict[key] = link

# Print the dictionary
for key, value in eci_links_dict.items():
    print(f"{key}: {value}")

PcResultGenJune2024: https://results.eci.gov.in/PcResultGenJune2024/index.htm
AcResultByeJune2024: https://results.eci.gov.in/AcResultByeJune2024/
AcResultGen2ndJune2024: https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm
AcResultGenJune2024: https://results.eci.gov.in/AcResultGenJune2024/index.htm


In [4]:
# Function to fetch state links for a given base_url
def fetch_state_links(base_url, state_codes):
    url = base_url
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all relevant links
    links = soup.find_all('a', href=True)
    partial_links = [link['href'] for link in links if any(state_code in link['href'] for state_code in state_codes)]
    
    # Complete the links by adding the base URL
    state_links = {}
    for link in partial_links:
        for state_code in state_codes:
            if state_code in link:
                state_name = get_state_name(state_code)
                state_links[state_name] = base_url + link
    
    return state_links

# Function to get state name from state code
def get_state_name(state_code):
    if state_code == "S01":
        return "Andhra Pradesh"
    elif state_code == "S02":
        return "Arunachal Pradesh"
    elif state_code == "S18":
        return "Odisha"
    elif state_code == "S21":
        return "Sikkim"
    else:
        return "Unknown"

# Base URLs
base_url_AcResultGen2ndJune2024 = "https://results.eci.gov.in/AcResultGen2ndJune2024/"
base_url_AcResultGenJune2024 = "https://results.eci.gov.in/AcResultGenJune2024/"

# State codes for each base URL
state_codes_AcResultGen2ndJune2024 = ["S02", "S21"]
state_codes_AcResultGenJune2024 = ["S01", "S18"]

# Fetch state links for AcResultGen2ndJune2024
state_links_dict_2ndJune2024 = fetch_state_links(base_url_AcResultGen2ndJune2024, state_codes_AcResultGen2ndJune2024)

# Fetch state links for AcResultGenJune2024
state_links_dict_June2024 = fetch_state_links(base_url_AcResultGenJune2024, state_codes_AcResultGenJune2024)

# Combine dictionaries
combined_state_links_dict = {**state_links_dict_2ndJune2024, **state_links_dict_June2024}

# Print the combined dictionary
combined_state_links_dict

{'Arunachal Pradesh': 'https://results.eci.gov.in/AcResultGen2ndJune2024/partywiseresult-S02.htm',
 'Sikkim': 'https://results.eci.gov.in/AcResultGen2ndJune2024/partywiseresult-S21.htm',
 'Andhra Pradesh': 'https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S01.htm',
 'Odisha': 'https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S18.htm'}

In [5]:
# URLs from the previous step
state_links_dict = {
    'Arunachal Pradesh': 'https://results.eci.gov.in/AcResultGen2ndJune2024/partywiseresult-S02.htm',
    'Sikkim': 'https://results.eci.gov.in/AcResultGen2ndJune2024/partywiseresult-S21.htm',
    'Andhra Pradesh': 'https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S01.htm',
    'Odisha': 'https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S18.htm'
}

# Function to fetch and convert links for each state
def fetch_and_convert_statewise_links(state_links_dict):
    updated_state_links_dict = {}

    for state, url in state_links_dict.items():
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the link
        link_element = soup.find('h3', class_='foot-title').find('a', href=True)
        if link_element:
            partial_link = link_element['href']
            complete_link = url.rsplit('/', 1)[0] + '/' + partial_link  # Complete the link
            updated_state_links_dict[state] = complete_link
    
    return updated_state_links_dict

# Fetch and convert statewise links
updated_state_links_dict = fetch_and_convert_statewise_links(state_links_dict)

# Print the updated dictionary
updated_state_links_dict

{'Arunachal Pradesh': 'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS021.htm',
 'Sikkim': 'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS211.htm',
 'Andhra Pradesh': 'https://results.eci.gov.in/AcResultGenJune2024/statewiseS011.htm',
 'Odisha': 'https://results.eci.gov.in/AcResultGenJune2024/statewiseS181.htm'}

In [6]:
response_ArPr1 = requests.get(updated_state_links_dict["Arunachal Pradesh"])
soup_ArPr1 = BeautifulSoup(response_ArPr1.content, 'html.parser')
table = soup_ArPr1.find("table", class_ = "table table-striped table-bordered")

In [7]:
headers = table.findAll("th")
titles = []
for i in headers:
    title = i.text
    titles.append(title)
print(titles)
df = pd.DataFrame(columns=titles[1:-1])
df

['Status Known For 60 out of 60 Constituencies', 'Constituency  ', 'Const. No.', 'Leading Candidate ', 'Leading Party ', 'Trailing Candidate ', 'Trailing Party ', 'Margin ', 'Round', 'Status', '\xa0']


,Constituency,Const. No.,Leading Candidate,Leading Party,Trailing Candidate,Trailing Party,Margin,Round,Status


In [8]:
table = soup.find('table')
# Find the <tbody> element under the <table>
tbody = table.find('tbody')
# Find all immediate <tr> children of <tbody>
rows = tbody.find_all('tr', recursive=False)

# rows = soup.find('tbody').find_all('tr', recursive=False)

# Initialize lists to store data
constituency_list = []
const_no_list = []
leading_candidate_list = []
leading_party_list = []
trailing_candidate_list = []
trailing_party_list = []
margin_list = []
round_list = []
status_list = []

# Iterate over each <tr> element
for tr in rows:
    # Initialize lists to store data for current row
    row_data = []
    # Find all <td> elements within the current <tr>
    for td in tr.find_all('td', recursive=False):
        # Check if <td> contains a <table>
        if td.find("table"):
            # Extract text from nested <tr> directly
            nested_tr_text = td.table.tbody.tr.td.get_text().strip()
            row_data.append(nested_tr_text)
        else:
            # If no <table> found, extract the text directly from <td>
            td_text = td.get_text().strip()
            row_data.append(td_text)
    
    # Append data to respective lists
    constituency_list.append(row_data[0])
    const_no_list.append(row_data[1])
    leading_candidate_list.append(row_data[2])
    leading_party_list.append(row_data[3])
    trailing_candidate_list.append(row_data[4])
    trailing_party_list.append(row_data[5])
    margin_list.append(row_data[6])
    round_list.append(row_data[7])
    status_list.append(row_data[8])

# Create DataFrame
df = pd.DataFrame({
    'Constituency': constituency_list,
    'Const. No.': const_no_list,
    'Leading Candidate': leading_candidate_list,
    'Leading Party': leading_party_list,
    'Trailing Candidate': trailing_candidate_list,
    'Trailing Party': trailing_party_list,
    'Margin': margin_list,
    'Round': round_list,
    'Status': status_list
})

# Display the DataFrame
df


AttributeError: 'NoneType' object has no attribute 'find'

In [14]:
response_ArPr1 = requests.get(updated_state_links_dict["Arunachal Pradesh"])
soup_ArPr1 = BeautifulSoup(response_ArPr1.content, 'html.parser')
table = soup_ArPr1.find("table", class_ = "table table-striped table-bordered")
headers = table.findAll("th")
titles = []
for i in headers:
    title = i.text
    titles.append(title)
print(titles)

table = soup.find('table')
# Find the <tbody> element under the <table>
tbody = table.find('tbody')
# Find all immediate <tr> children of <tbody>
rows = tbody.find_all('tr', recursive=False)

# rows = soup.find('tbody').find_all('tr', recursive=False)

# Initialize lists to store data
constituency_list = []
const_no_list = []
leading_candidate_list = []
leading_party_list = []
trailing_candidate_list = []
trailing_party_list = []
margin_list = []
round_list = []
status_list = []

# Iterate over each <tr> element
for tr in rows:
    # Initialize lists to store data for current row
    row_data = []
    # Find all <td> elements within the current <tr>
    for td in tr.find_all('td', recursive=False):
        # Check if <td> contains a <table>
        if td.find("table"):
            # Extract text from nested <tr> directly
            nested_tr_text = td.table.tbody.tr.td.get_text().strip()
            row_data.append(nested_tr_text)
        else:
            # If no <table> found, extract the text directly from <td>
            td_text = td.get_text().strip()
            row_data.append(td_text)
    
    # Append data to respective lists
    constituency_list.append(row_data[0])
    const_no_list.append(row_data[1])
    leading_candidate_list.append(row_data[2])
    leading_party_list.append(row_data[3])
    trailing_candidate_list.append(row_data[4])
    trailing_party_list.append(row_data[5])
    margin_list.append(row_data[6])
    round_list.append(row_data[7])
    status_list.append(row_data[8])

# Create DataFrame
df = pd.DataFrame({
    'Constituency': constituency_list,
    'Const. No.': const_no_list,
    'Leading Candidate': leading_candidate_list,
    'Leading Party': leading_party_list,
    'Trailing Candidate': trailing_candidate_list,
    'Trailing Party': trailing_party_list,
    'Margin': margin_list,
    'Round': round_list,
    'Status': status_list
})

# Display the DataFrame
df


['Status Known For 60 out of 60 Constituencies', 'Constituency  ', 'Const. No.', 'Leading Candidate ', 'Leading Party ', 'Trailing Candidate ', 'Trailing Party ', 'Margin ', 'Round', 'Status', '\xa0']


,Constituency,Const. No.,Leading Candidate,Leading Party,Trailing Candidate,Trailing Party,Margin,Round,Status
0,Uravakonda,149,PAYYAVULA KESHAV,Telugu Desam,Y. VISWESWARA REDDY,Yuvajana Sramika Rythu Congress Party,21704,15/15,Result Declared
1,Vemuru (SC),89,ANANDA BABU NAKKA,Telugu Desam,ASHOK BABU VARIKUTI,Yuvajana Sramika Rythu Congress Party,22021,17/17,Result Declared
2,Venkatagiri,122,KURUGONDLA RAMAKRISHNA,Telugu Desam,NEDURUMALLI RAM KUMAR REDDY,Yuvajana Sramika Rythu Congress Party,16294,22/22,Result Declared
3,Vijayawada Central,80,BONDA UMAMAHESWARARAO,Telugu Desam,VELAMPALLI SRINIVASA RAO,Yuvajana Sramika Rythu Congress Party,68886,21/21,Result Declared
4,Vijayawada East,81,GADDE RAMA MOHAN,Telugu Desam,AVINASH DEVINENI,Yuvajana Sramika Rythu Congress Party,49640,23/23,Result Declared
5,Vijayawada West,79,YALAMANCHILI SATYANARAYANA CHOWDARY (SUJANA CH...,Bharatiya Janata Party,ASIF SHAIK,Yuvajana Sramika Rythu Congress Party,47032,20/20,Result Declared
6,Vinukonda,99,GONUGUNTLA VENKATA SIVA SITA RAMA ANZANNEYLLU,Telugu Desam,BOLLA BRAHMA NAIDU,Yuvajana Sramika Rythu Congress Party,30267,22/22,Result Declared
7,Visakhapatnam East,21,RAMAKRISHNA BABU VELAGAPUDI,Telugu Desam,M V V SATYANARAYANA,Yuvajana Sramika Rythu Congress Party,70877,22/22,Result Declared
8,Visakhapatnam North,23,VISHNU KUMAR RAJU PENMETSA,Bharatiya Janata Party,KANNAPARAJU KAMMILA (K.K. RAJU),Yuvajana Sramika Rythu Congress Party,47534,21/21,Result Declared
9,Visakhapatnam South,22,CH.VAMSI KRISHNA SRINIVAS,Janasena Party,GANESH KUMAR VASUPALLI,Yuvajana Sramika Rythu Congress Party,64594,17/17,Result Declared


<h1>Arunachal Pradesh General Elections

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URLs for each state's result pages
state_links = [
    'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS021.htm',
    'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS022.htm',
    'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS023.htm'
]

# Initialize lists to store data
all_data = []

# Iterate over each state link
for url in state_links:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table containing election results
    table = soup.find("table", class_="table table-striped table-bordered")
    
    # Find all rows in the table body
    rows = table.find('tbody').find_all('tr', recursive=False)
    
    # Iterate over each row
    for tr in rows:
        # Initialize a dictionary to store row data
        row_data = {}
        
        # Find all <td> elements within the current <tr>
        columns = tr.find_all('td', recursive=False)
        
        # Extract data from each column (td element)
        for index, td in enumerate(columns):
            # Check if <td> contains a <table>
            if td.find("table"):
                # Extract text from nested <tr> directly
                nested_tr_text = td.table.tbody.tr.td.get_text().strip()
                row_data[f'Column {index + 1}'] = nested_tr_text
            else:
                # If no <table> found, extract the text directly from <td>
                td_text = td.get_text(strip=True)
                row_data[f'Column {index + 1}'] = td_text
        
        # Append the row data to the list of all data
        all_data.append(row_data)

# Create a DataFrame from the collected data
df = pd.DataFrame(all_data)

# Display the DataFrame
df


,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7,Column 8,Column 9
0,Along (East),31,KENTO JINI,Bharatiya Janata Party,JARKAR GAMLIN,People's Party of Arunachal,3158,4/4,Result Declared
1,Along (West),30,TOPIN ETE,Bharatiya Janata Party,NYAMO ETE,National People's Party,1951,4/4,Result Declared
2,Anini,41,MOPI MIHU,Bharatiya Janata Party,ERI TAYU,Independent,1173,5/5,Result Declared
3,Bameng,8,KUMAR WAII,Indian National Congress,DOBA LAMNIO,Bharatiya Janata Party,635,4/4,Result Declared
4,Basar,29,NYABI JINI DIRCHI,Bharatiya Janata Party,GOKAR BASAR,National People's Party,1791,5/5,Result Declared
5,Bomdila,7,DONGRU SIONGJU,Bharatiya Janata Party,,,-,-,Uncontested
6,Bordumsa-Diyun,49,NIKH KAMIN,Nationalist Congress Party,SOMLUNG MOSSANG,Bharatiya Janata Party,1352,5/5,Result Declared
7,Borduria-Bogapani,57,WANGLIN LOWANGDONG,Bharatiya Janata Party,JOWANG HOSAI,Nationalist Congress Party,1452,3/3,Result Declared
8,Changlang(North),53,TESAM PONGTE,Bharatiya Janata Party,DIHOM KITNYA,National People's Party,2002,6/6,Result Declared
9,Changlang(South),52,HAMJONG TANGHA,Bharatiya Janata Party,TIMPU NGEMU,National People's Party,1482,6/6,Result Declared


<h1>Andhra Pradesh General Elections

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URLs for Andhra Pradesh result pages
ap_links = [
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS011.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS012.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS013.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS014.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS015.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS016.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS017.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS018.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS019.htm'
]

# Initialize lists to store data
ap_gen_data = []

# Iterate over each AP state link
for url in ap_links:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table containing election results
    table = soup.find("table", class_="table table-striped table-bordered")
    
    # Find all rows in the table body
    rows = table.find('tbody').find_all('tr', recursive=False)
    
    # Iterate over each row
    for tr in rows:
        # Initialize a dictionary to store row data
        row_data = {}
        
        # Find all <td> elements within the current <tr>
        columns = tr.find_all('td', recursive=False)
        
        # Extract data from each column (td element)
        for index, td in enumerate(columns):
            # Check if <td> contains a <table>
            if td.find("table"):
                # Extract text from nested <tr> directly
                nested_tr_text = td.table.tbody.tr.td.get_text().strip()
                row_data[f'Column {index + 1}'] = nested_tr_text
            else:
                # If no <table> found, extract the text directly from the <td>
                td_text = td.get_text(strip=True)
                row_data[f'Column {index + 1}'] = td_text
        
        # Append the row data to the list of AP data
        ap_gen_data.append(row_data)

# Create a DataFrame for Andhra Pradesh election results
ap_gen = pd.DataFrame(ap_gen_data)

# Display the DataFrame
ap_gen


,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7,Column 8,Column 9
0,Achanta,56,SATYANARAYANA PITHANI,Telugu Desam,CHERUKUVADA SRI RANGANADHA RAJU,Yuvajana Sramika Rythu Congress Party,26554,14/14,Result Declared
1,Addanki,105,GOTTIPATI RAVI KUMAR,Telugu Desam,CHINNA HANIMIREDDY PANEM,Yuvajana Sramika Rythu Congress Party,24890,22/22,Result Declared
2,Adoni,146,DR. PARTHA SARATHI VALMIKI,Bharatiya Janata Party,Y. SAI PRASAD REDDY,Yuvajana Sramika Rythu Congress Party,18164,19/19,Result Declared
3,Allagadda,134,AKHILA PRIYA BHUMA,Telugu Desam,BRIJENDRA REDDY GANGULA (NANI),Yuvajana Sramika Rythu Congress Party,12037,22/22,Result Declared
4,Alur,147,B. VIRUPAKSHI,Yuvajana Sramika Rythu Congress Party,B. VEERABHADRA GOWD,Telugu Desam,2831,21/21,Result Declared
...,...,...,...,...,...,...,...,...,...
170,Visakhapatnam West,24,P.G.V.R.NAIDU(GANABABU),Telugu Desam,ADARI ANAND KUMAR,Yuvajana Sramika Rythu Congress Party,35184,16/16,Result Declared
171,Vizianagaram,18,ADITI VIJAYALAKSHMI GAJAPATHI RAJU PUSAPATI,Telugu Desam,KOLAGATLA VEERABHADRA SWAMY,Yuvajana Sramika Rythu Congress Party,60609,20/20,Result Declared
172,Yelamanchili,32,SUNDARAPU VIJAY KUMAR,Janasena Party,UPPALAPATI VENKATA RAMANAMURTHY RAJU,Yuvajana Sramika Rythu Congress Party,48956,19/19,Result Declared
173,Yemmiganur,144,B. JAYANAGESWARA REDDY,Telugu Desam,BUTTA RENUKA,Yuvajana Sramika Rythu Congress Party,15837,20/20,Result Declared


<h1>Odisha General Elections

In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URLs for Odisha result pages
odisha_links = [
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS181.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS182.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS183.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS184.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS185.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS186.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS187.htm',
    'https://results.eci.gov.in/AcResultGenJune2024/statewiseS188.htm'
]

# Initialize lists to store data
odisha_gen_data = []

# Iterate over each Odisha state link
for url in odisha_links:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table containing election results
    table = soup.find("table", class_="table table-striped table-bordered")
    
    # Find all rows in the table body
    rows = table.find('tbody').find_all('tr', recursive=False)
    
    # Iterate over each row
    for tr in rows:
        # Initialize a dictionary to store row data
        row_data = {}
        
        # Find all <td> elements within the current <tr>
        columns = tr.find_all('td', recursive=False)
        
        # Extract data from each column (td element)
        for index, td in enumerate(columns):
            # Check if <td> contains a <table>
            if td.find("table"):
                # Extract text from nested <tr> directly
                nested_tr_text = td.table.tbody.tr.td.get_text().strip()
                row_data[f'Column {index + 1}'] = nested_tr_text
            else:
                # If no <table> found, extract the text directly from the <td>
                td_text = td.get_text(strip=True)
                row_data[f'Column {index + 1}'] = td_text
        
        # Append the row data to the list of Odisha data
        odisha_gen_data.append(row_data)

# Create a DataFrame for Odisha election results
odisha_gen = pd.DataFrame(odisha_gen_data)

# Display the DataFrame
odisha_gen


,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7,Column 8,Column 9
0,Anandapur,22,ABHIMANYU SETHI,Biju Janata Dal,JAYADEV JENA,Indian National Congress,10966,25/25,Result Declared
1,Angul,61,PRATAP CHANDRA PRADHAN,Bharatiya Janata Party,SANJUKTA SINGH,Biju Janata Dal,17433,21/21,Result Declared
2,Aska,128,SAROJ KUMAR PADHI,Bharatiya Janata Party,MANJULA SWAIN,Biju Janata Dal,8059,15/15,Result Declared
3,Athagarh,89,RANENDRA PRATAP SWAIN,Biju Janata Dal,ABHAYA KUMAR BARIK,Bharatiya Janata Party,3584,19/19,Result Declared
4,Athamallik,63,NALINI KANTA PRADHAN,Biju Janata Dal,SANJEEB KUMAR SAHOO,Bharatiya Janata Party,16153,21/21,Result Declared
...,...,...,...,...,...,...,...,...,...
142,Telkoi,20,DR. FAKIR MOHAN NAIK,Bharatiya Janata Party,MADHABA SARDAR,Biju Janata Dal,9439,23/23,Result Declared
143,Tirtol,102,RAMAKANTA BHOI,Biju Janata Dal,RAJKISHORE BEHERA,Bharatiya Janata Party,28495,20/20,Result Declared
144,Titlagarh,69,NABIN KUMAR JAIN,Bharatiya Janata Party,TUKUNI SAHU,Biju Janata Dal,17399,23/23,Result Declared
145,Udala,31,BHASKAR MADHEI,Bharatiya Janata Party,SRINATH SOREN,Biju Janata Dal,6517,21/21,Result Declared


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URLs for Sikkim result pages
sikkim_links = [
    'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS211.htm',
    'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS212.htm'
]

# Initialize lists to store data
sikkim_gen_data = []

# Iterate over each Sikkim state link
for url in sikkim_links:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table containing election results
    table = soup.find("table", class_="table table-striped table-bordered")
    
    # Find all rows in the table body
    rows = table.find('tbody').find_all('tr', recursive=False)
    
    # Iterate over each row
    for tr in rows:
        # Initialize a dictionary to store row data
        row_data = {}
        
        # Find all <td> elements within the current <tr>
        columns = tr.find_all('td', recursive=False)
        
        # Extract data from each column (td element)
        for index, td in enumerate(columns):
            # Check if <td> contains a <table>
            if td.find("table"):
                # Extract text from nested <tr> directly
                nested_tr_text = td.table.tbody.tr.td.get_text().strip()
                row_data[f'Column {index + 1}'] = nested_tr_text
            else:
                # If no <table> found, extract the text directly from the <td>
                td_text = td.get_text(strip=True)
                row_data[f'Column {index + 1}'] = td_text
        
        # Append the row data to the list of Sikkim data
        sikkim_gen_data.append(row_data)

# Create a DataFrame for Sikkim election results
sikkim_gen = pd.DataFrame(sikkim_gen_data)

# Display the DataFrame
sikkim_gen


,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7,Column 8,Column 9
0,ARITHANG,26,ARUN KUMAR UPRETI,Sikkim Krantikari Morcha,ASHIS RAI,Sikkim Democratic Front,2729,3/3,Result Declared
1,BARFUNG,9,RIKSHAL DORJEE BHUTIA,Sikkim Krantikari Morcha,BHAICHUNG BHUTIA,Sikkim Democratic Front,4346,3/3,Result Declared
2,CHUJACHEN,20,PURAN KR. GURUNG,Sikkim Krantikari Morcha,MANI KUMAR GURUNG,Sikkim Democratic Front,3334,4/4,Result Declared
3,DARAMDIN,6,MINGMA NORBU SHERPA,Sikkim Krantikari Morcha,PEM NORBU SHERPA,Sikkim Democratic Front,5975,3/3,Result Declared
4,DJONGU,30,PINTSO NAMGYAL LEPCHA,Sikkim Krantikari Morcha,SONAM GYATSO LEPCHA,Sikkim Democratic Front,5007,5/5,Result Declared
5,GANGTOK,27,DELAY NAMGYAL BARFUNGPA,Sikkim Krantikari Morcha,PINTSO CHOPEL LEPCHA,Sikkim Democratic Front,2692,3/3,Result Declared
6,GNATHANG-MACHONG,21,PAMIN LEPCHA,Sikkim Krantikari Morcha,TSHERING WANGDI LEPCHA,Sikkim Democratic Front,3807,3/3,Result Declared
7,GYALSHING-BARNYAK,4,LOK NATH SHARMA,Sikkim Krantikari Morcha,KHUSANDRA PRASAD SHARMA,Independent,963,3/3,Result Declared
8,KABI LUNGCHUK,29,THENLAY TSHERING BHUTIA,Sikkim Krantikari Morcha,GNAWO CHOPEL LEPCHA,Sikkim Democratic Front,1693,4/4,Result Declared
9,KHAMDONG-SINGTAM,17,NAR BAHADUR DAHAL,Sikkim Krantikari Morcha,MANI KUMAR SHARMA,Sikkim Democratic Front,1739,3/3,Result Declared


In [51]:
pd.DataFrame(pd.read_csv('LS2024.csv',delimiter=';'))


,State,Constituency,Party,Candidate,Votes,State ID,Constituency ID
0,Andhra Pradesh,Araku,Yuvajana Sramika Rythu Congress Party ...,GUMMA THANUJA RANI ...,477005,S01,1
1,Andhra Pradesh,Araku,Bharatiya Janata Party ...,KOTHAPALLI GEETHA ...,426425,S01,1
2,Andhra Pradesh,Araku,Communist Party of India (Marxist) ...,APPALANARASA PACHIPENTA ...,123129,S01,1
3,Andhra Pradesh,Araku,Bahujan Samaj Party ...,AVASHYA LAHARI . VARAM ...,25750,S01,1
4,Andhra Pradesh,Araku,Independent ...,SAMAREDDY BALAKRISHNA ...,9535,S01,1
...,...,...,...,...,...,...,...
8897,Telangana,Khammam,Jai Bharat National Party ...,BHATTU SRINIVAS ...,476,S29,1
8898,Telangana,Khammam,Independent ...,NAGESWARA RAO LAKAVATH ...,368,S29,1
8899,Telangana,Khammam,Independent ...,MALAMPATI SURYANARAYANA ...,363,S29,1
8900,Telangana,Khammam,Independent ...,MUTYAM ARJUNA RAJU ...,351,S29,1


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URLs categorized by state
state_urls = {
    'Arunachal Pradesh': [
        'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS021.htm',
        'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS022.htm',
        'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS023.htm'
    ],
    'Andhra Pradesh': [
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS011.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS012.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS013.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS014.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS015.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS016.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS017.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS018.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS019.htm'
    ],
    'Odisha': [
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS181.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS182.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS183.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS184.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS185.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS186.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS187.htm',
        'https://results.eci.gov.in/AcResultGenJune2024/statewiseS188.htm'
    ],
    'Sikkim': [
        'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS211.htm',
        'https://results.eci.gov.in/AcResultGen2ndJune2024/statewiseS212.htm'
    ]
}

# Function to fetch data from a given URL and parse it into a DataFrame
def fetch_election_data(url, state):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table containing election results
    table = soup.find("table", class_="table table-striped table-bordered")
    
    if not table:
        return pd.DataFrame()  # Return an empty DataFrame if no table is found

    # Find all rows in the table body
    rows = table.find('tbody').find_all('tr', recursive=False)
    
    # Initialize a list to store row data
    all_data = []
    
    # Iterate over each row
    for tr in rows:
        # Initialize a dictionary to store row data
        row_data = {'State': state}
        
        # Find all <td> elements within the current <tr>
        columns = tr.find_all('td', recursive=False)
        
        # Extract data from each column (td element)
        for index, td in enumerate(columns):
            # Check if <td> contains a <table>
            if td.find("table"):
                # Extract text from nested <tr> directly
                nested_tr_text = td.table.tbody.tr.td.get_text().strip()
                row_data[f'Column {index + 1}'] = nested_tr_text
            else:
                # If no <table> found, extract the text directly from <td>
                td_text = td.get_text(strip=True)
                row_data[f'Column {index + 1}'] = td_text
        
        # Append the row data to the list of all data
        all_data.append(row_data)
    
    # Create a DataFrame from the collected data
    df = pd.DataFrame(all_data)
    return df

# Initialize an empty DataFrame to store combined data
combined_data = pd.DataFrame()

# Iterate over each state and their respective URLs
for state, urls in state_urls.items():
    for url in urls:
        state_data = fetch_election_data(url, state)
        combined_data = pd.concat([combined_data, state_data], ignore_index=True)


,State,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7,Column 8,Column 9
0,Arunachal Pradesh,Along (East),31,KENTO JINI,Bharatiya Janata Party,JARKAR GAMLIN,People's Party of Arunachal,3158,4/4,Result Declared
1,Arunachal Pradesh,Along (West),30,TOPIN ETE,Bharatiya Janata Party,NYAMO ETE,National People's Party,1951,4/4,Result Declared
2,Arunachal Pradesh,Anini,41,MOPI MIHU,Bharatiya Janata Party,ERI TAYU,Independent,1173,5/5,Result Declared
3,Arunachal Pradesh,Bameng,8,KUMAR WAII,Indian National Congress,DOBA LAMNIO,Bharatiya Janata Party,635,4/4,Result Declared
4,Arunachal Pradesh,Basar,29,NYABI JINI DIRCHI,Bharatiya Janata Party,GOKAR BASAR,National People's Party,1791,5/5,Result Declared
...,...,...,...,...,...,...,...,...,...,...
409,Sikkim,UPPER BURTUK,28,KALA RAI,Sikkim Krantikari Morcha,DILLI RAM THAPA,Bharatiya Janata Party,2568,3/3,Result Declared
410,Sikkim,UPPER TADONG,25,G.T. DHUNGEL,Sikkim Krantikari Morcha,DR. CHANDRA BAHADUR CHETTRI,Sikkim Democratic Front,4089,4/4,Result Declared
411,Sikkim,WEST PENDAM,18,LALL BAHADUR DAS,Sikkim Krantikari Morcha,ANUP THATAL,Sikkim Democratic Front,1952,3/3,Result Declared
412,Sikkim,YANGTHANG,2,BHIM HANG LIMBOO,Sikkim Krantikari Morcha,KESHAM LIMBOO,Sikkim Democratic Front,2556,3/3,Result Declared


In [23]:
combined_data.columns = [
    'State', 'Constituency', 'Const. No.', 'Leading Candidate', 'Leading Party',
    'Trailing Candidate', 'Trailing Party', 'Margin', 'Round', 'Status'
]

# Display the combined DataFrame
combined_data

,State,Constituency,Const. No.,Leading Candidate,Leading Party,Trailing Candidate,Trailing Party,Margin,Round,Status
0,Arunachal Pradesh,Along (East),31,KENTO JINI,Bharatiya Janata Party,JARKAR GAMLIN,People's Party of Arunachal,3158,4/4,Result Declared
1,Arunachal Pradesh,Along (West),30,TOPIN ETE,Bharatiya Janata Party,NYAMO ETE,National People's Party,1951,4/4,Result Declared
2,Arunachal Pradesh,Anini,41,MOPI MIHU,Bharatiya Janata Party,ERI TAYU,Independent,1173,5/5,Result Declared
3,Arunachal Pradesh,Bameng,8,KUMAR WAII,Indian National Congress,DOBA LAMNIO,Bharatiya Janata Party,635,4/4,Result Declared
4,Arunachal Pradesh,Basar,29,NYABI JINI DIRCHI,Bharatiya Janata Party,GOKAR BASAR,National People's Party,1791,5/5,Result Declared
...,...,...,...,...,...,...,...,...,...,...
409,Sikkim,UPPER BURTUK,28,KALA RAI,Sikkim Krantikari Morcha,DILLI RAM THAPA,Bharatiya Janata Party,2568,3/3,Result Declared
410,Sikkim,UPPER TADONG,25,G.T. DHUNGEL,Sikkim Krantikari Morcha,DR. CHANDRA BAHADUR CHETTRI,Sikkim Democratic Front,4089,4/4,Result Declared
411,Sikkim,WEST PENDAM,18,LALL BAHADUR DAS,Sikkim Krantikari Morcha,ANUP THATAL,Sikkim Democratic Front,1952,3/3,Result Declared
412,Sikkim,YANGTHANG,2,BHIM HANG LIMBOO,Sikkim Krantikari Morcha,KESHAM LIMBOO,Sikkim Democratic Front,2556,3/3,Result Declared
